In [ ]:
!pip install datasets
!pip install transformers
!pip install sentencepiece
!pip install transformers[torch]
!pip install sacrebleu
!pip install evaluate
!pip install sacrebleu
!pip install accelerate -U
!pip install gradio
!pip install kaleido cohere openai tiktoken

  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
Using cached typing_extensions-4.15.0-py3-none-any.whl (44 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastapi 0.128.0 requires pydantic>=2.7.0, but you have pydantic 1.10.26 which is incompatible.
mlflow-skinny 3.8.1 requires pydantic<3,>=2.0.0, but you have pydantic 1.10.26 which is incompatible.
mlflow-tracing 3.8.1 requires pydantic<3,>=2.0.0, but you have pydantic 1.10.26 which is incompatible.
gradio 6.3.0 requires pydantic<=3.0,>=2.0, but you have pydantic 1.10.26 which is incompatible.
ydata-profiling 4.18.0 requires pydantic<3,>=2, but you have pydantic 1.10.26 which is incompat

In [3]:
from datasets import load_dataset
dataset=load_dataset("cfilt/iitb-english-hindi")

TypeError: type 'typing.TypeVar' is not an acceptable base type

In [ ]:
max_length=256
from transformers import AutoTokenizer,AutoModelForSeq2SeqLM
tokenizer=AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-hi")
model=AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-hi")

In [ ]:
article=dataset['validation'][2]['translation']['en']
inputs=tokenizer.encode(article,return_tensors='pt')